# Cubic splines

The file `cubic_splines.pdf` presents some notes about the cubic splines method.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange

#used for inserting images
from IPython.display import Image as img

In [ ]:
L = 5
x = np.linspace(0., 10., L) # x coordinates of the data
y = np.sin(2.*np.pi*x/10.)  # data values

In [ ]:
x0 = np.linspace(0., 10., 5+3*4) # x coordinates of the interpolating points
y0 = np.sin(2.*np.pi*x0/10.)   # true values of the original function at the interpolation points

In [ ]:
plt.figure(figsize=(10,10))
plt.axis('scaled')
plt.plot(x, y, 'bo', label='Data')
plt.plot(x0, y0, 'b.', label='True interpolated data')
plt.xlim(np.min(x0), np.max(x0))
plt.ylim(-2., 2.)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='best', numpoints=1, fontsize=16)
plt.show()

In [ ]:
def spline3(x, f):
    
    L = f.size
    
    g = 3.*np.hstack(((f[1] - f[0]), (f[2:] - f[:L-2]), (f[-1] - f[-2])))

    indices = np.arange(L)
    A = np.zeros((L,L))
    A[indices[1:L-1],indices[1:L-1]] = 4
    A[0,0] = 2
    A[L-1,L-1] = 2
    A[indices[:L-1],indices[1:]] = 1
    A[indices[1:],indices[:L-1]] = 1
    
    D = np.linalg.solve(np.dot(A.T, A), np.dot(A.T, g))
    
    c = 3.*(f[1:] - f[:L-1]) - 2.*D[:L-1] - D[1:]
    d = 2.*(f[:L-1] - f[1:]) + D[:L-1] + D[1:]
    
    return D, c, d

In [ ]:
def spline3_interpolate(a,b,c,d,t,x):
    
    y0 = np.empty((a.size,t.size))
    
    e = t.copy()
    
    for i, ai in enumerate(a):
        y0[i] = ai
    for i, bi in enumerate(b):
        y0[i] += bi*e
    e = e*t
    for i, ci in enumerate(c):
        y0[i] += ci*e
    e = e*t
    for i, di in enumerate(d):
        y0[i] += di*e
   
    y0 = np.ravel(y0)
    
    x0 = np.empty((a.size,t.size))
    for i in range(a.size):
        dx = x[i+1] - x[i]
        x0[i] = x[i] + dx*t
    x0 = np.ravel(x0)
    
    return x0, y0

In [ ]:
def spline3_interpolate2(a,b,c,d,x,x0):
    
    L = x.size
    
    indices = np.arange(x0.size)
    split = []
    i0 = 0
    
    for i, xi, xi_plus in enumerate(x[:L-1],x[1:]):
        mask = (x0[i0:] >= xi) and (x0[i0:] < xi_plus)
        split.append(indices[mask])
        i0 = indices[mask][-1] + 1
        indices = indices[mask]
    
    for i, ai in enumerate(a):
        y0[i] = ai
    for i, bi in enumerate(b):
        y0[i] += bi*e
    e = e*t
    for i, ci in enumerate(c):
        y0[i] += ci*e
    e = e*t
    for i, di in enumerate(d):
        y0[i] += di*e
   
    y0 = np.ravel(y0)
    
    x0 = np.empty((a.size,t.size))
    for i in range(a.size):
        dx = x[i+1] - x[i]
        x0[i] = x[i] + dx*t
    x0 = np.ravel(x0)
    
    return x0, y0

In [43]:
x0 = np.sort(np.hstack((np.random.rand(10)*10, np.arange(11))))
print x0

x = np.arange(0.,11.,2.)
print x

L = x.size

xmax = np.max(x)
xmin = np.min(x)

x0_norm = x0*(x.size-1)/(xmax - xmin)
print x0_norm

x_norm = x*(x.size-1)/(xmax - xmin)
print x_norm

split = []
i0 = 0
for i, xi_plus in enumerate(x_norm[1:]):
    print i, xi_plus
    mask = x0_norm[i0:] < xi_plus
    split.append(x0_norm[i0:][mask]-i)
    i0 += x0_norm[i0:][mask].size

for sp in split:
    print sp

[  0.           0.34083183   1.           1.06643063   1.90270109   2.
   2.05689567   2.87587031   3.           3.01571644   3.23953361   4.           5.
   6.           6.31129322   7.           7.68897821   8.           9.
   9.80198743  10.        ]
[  0.   2.   4.   6.   8.  10.]
[ 0.          0.17041592  0.5         0.53321531  0.95135054  1.
  1.02844783  1.43793516  1.5         1.50785822  1.6197668   2.          2.5
  3.          3.15564661  3.5         3.8444891   4.          4.5
  4.90099371  5.        ]
[ 0.  1.  2.  3.  4.  5.]
0 1.0
1 2.0
2 3.0
3 4.0
4 5.0
[ 0.          0.17041592  0.5         0.53321531  0.95135054]
[ 0.          0.02844783  0.43793516  0.5         0.50785822  0.6197668 ]
[ 0.   0.5]
[ 0.          0.15564661  0.5         0.8444891 ]
[ 0.          0.5         0.90099371]


In [18]:
L = x.size
L0 = x0.size
    
indices = np.arange(x0.size)
split = []
i0 = 0
print i0
print indices

for i, (xi, xi_plus) in enumerate(zip(x[:L-1],x[1:])):
    print i
    print xi, xi_plus
    mask = (x0[i0:] >= xi) & (x0[i0:] < xi_plus)
    split.append(indices[mask])
    i0 = indices[mask][-1] + 1
    if i0 >= L0:
        break
    print i0
    indices = np.delete(indices, 
    print indices
    x0 = x0[i0:]
    print x0

0
[ 0  1  2  3  4  5  6  7  8  9 10 11]
0
1.0 4.0
2
[ 2  3  4  5  6  7  8  9 10 11]
[  4.20701833   4.92718984   5.06562716   5.88432496   5.90750611
   7.65855465   8.38016733   8.88389164   9.96866649  10.5468182 ]
1
4.0 7.0
5
[ 7  8  9 10 11]
[  7.65855465   8.38016733   8.88389164   9.96866649  10.5468182 ]
2
7.0 10.0


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 10 but corresponding boolean dimension is 8
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 10 but corresponding boolean dimension is 8


IndexError: index -1 is out of bounds for axis 0 with size 0

In [15]:
for sp in split:
    print sp

[0 1]
[2 3 4 5 6]


In [ ]:
b,c,d = spline3(x,y)

In [ ]:
t = np.linspace(0.25, 0.75, 3)

In [ ]:
x0_calc, y0_calc = spline3_interpolate(y[:L-1],b[:L-1],c,d,t,x)

In [ ]:
plt.figure(figsize=(10,10))
plt.axis('scaled')
plt.plot(x, y, 'bo', label='Data')
plt.plot(x0, y0, 'b.', label='True interpolated data')
plt.plot(x0_calc, y0_calc, 'xr', label='Interpolated data')
plt.xlim(np.min(x0), np.max(x0))
plt.ylim(-2., 2.)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='best', numpoints=1, fontsize=16)
plt.show()

### Exercise 25

The folder `synthetic_gravity_data` contains a jupyter notebook named [`synthetic_data_fatiando.ipynb`](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/synthetic_gravity_data/synthetic_data_fatiando.ipynb). This code uses the open-source Python toolkit for geophysical modeling and inversion [Fatiando a Terra](http://www.fatiando.org/) for calculating the gravity anomaly produced by the following synthetic body:

In [ ]:
img('synthetic_gravity_data/model.png')

The synthetic body shown above produces the gravity anomaly shown below:

In [ ]:
img('synthetic_gravity_data/map.png')

In [ ]:
img('synthetic_gravity_data/profile.png')

The files `synthetic_gravity_data/gravity_anomaly.txt` and `synthetic_gravity_data/gravity_anomaly_profile.txt` contain, respectively, the synthetic gravity anomaly map and the gravity anomaly profile on x = 0 m.

To solve this exercise:

1. Load the file `synthetic_gravity_data/gravity_anomaly_profile.txt` by using the routine [`numpy.loadtxt`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html). 

2. Use the function `my_neville` to calculate one interpolated gravity data between two adjacent gravity data (black dots on the gravity profile) on the profile.

3. Plot a Figure similar to the gravity profile shown above containing: (i) the original data (black dots on the gravity profile) and (ii) the interpolated points.